In [10]:
from matplotlib import pyplot as plt
import time
import numpy as np
import scipy as sp
from scipy import interpolate
import pandas as pd
import geopandas as gpd
import LMIPy
import xarray as xr
import dask.array as da
from dask.diagnostics import ProgressBar
#import regionmask
import zarr
import gcsfs
import requests
import shapely.wkb 
from shapely.ops import cascaded_union
import regionmask
from tqdm import tqdm

In [11]:
def df_from_carto(account, query):
    """
    It gets data by querying a carto table and converts it into a GeoDataFrame.
    """
    urlCarto = f"https://{account}.carto.com/api/v2/sql"
    
    sql = {"q": query}
    r = requests.get(urlCarto, params=sql)
    
    data = r.json()
    
    df = gpd.GeoDataFrame(data.get("rows"))
    if 'the_geom' in df.columns:
        # Change geometry from WKB to WKT format
        df['geometry'] = df.apply(lambda x: shapely.wkb.loads(x['the_geom'],hex=True), axis=1 )
        df.drop(columns=['the_geom'], inplace=True)
        if 'the_geom_webmercator' in df.columns:
            df.drop(columns=['the_geom_webmercator'], inplace=True)
        df.crs = {'init': 'epsg:4326'}
        df = df.to_crs({'init': 'epsg:4326'})
        
    return df

def weighted_mean(df, columns=['cfr_raw'], weight_column='area', groupby_on='gid_2'):
    df = df.copy()
    
    # Define a lambda function to compute the weighted mean:
    wm = lambda x: np.average(x, weights=df.loc[x.index, weight_column])

    # Groupby and aggregate with lambda function:
    for n, column in enumerate(columns):
        if n == 0:
            df_new = df.groupby(groupby_on).agg({column: wm})  
        else:
            df_new[column] = df.groupby(groupby_on).agg({column: wm})[column]
        
    return gpd.GeoDataFrame(df_new)


def quantile_interp_function(s,q,y):
    """ Get a interpolated function based on quantiles.
    y and q should be the same length.
    
    Args:
        s(pandas Series): Input y data that needs to 
            be remapped.
        q(list): list with quantile x values.
        y(list): list with y value to map to.
        
    Returns:
        f(interp1d) : Scipy function object.
        quantiles(Pandas Series): list of quantile y 
            values. 
        
    Example:
    
        s = df["col"]
        q = [0,0.2,0.4,0.6,0.8,1]
        y = [0,1,2,3,4,5]
        f = quantile_interp_function(s,quantiles,y)
        y_new = f(x)
    
    """
    quantiles = s.quantile(q=q)
    print("quantiles used for aggregate total:",quantiles)
    f = interpolate.interp1d(quantiles,y)
    return f, quantiles

def score_to_category(score):
    if np.isnan(score):
        cat = -1
    elif score != 5:
        cat = int(np.floor(score))
    else:
        cat = 4
    return cat

def decile_to_category(decile):
    if np.isnan(decile):
        cat = -1
    elif (decile >= 0) and (decile <= 2):
        cat = 0
    elif (decile > 2) and (decile <= 4):
        cat = 1
    elif (decile > 4) and (decile <= 6):
        cat = 2
    elif (decile > 6) and (decile <= 8):
        cat = 3
    elif (decile > 8) and (decile <= 10):
        cat = 4
    else:
        cat = 4
    return cat

def compute_score_category_label(df, columns, q, scores):
    df = df.copy()
    for column in columns:
        df.sort_values(column, inplace=True)
        
        # Geometries without a hazard are given the lowest risk score, 0
        df_0 = df[df[column] == 0]
        df_0[f"{column}_score"] = 0.
        
        df = df[df[column] != 0]
        s = df[column]
        f, quantiles =quantile_interp_function(s,q,scores)

        # Add scores
        df[f"{column}_score"] = df[column].apply(f)
        
        df = pd.concat([df_0, df])
                                                          
        # Add categories
        df[f"{column}_cat"] = df[f"{column}_score"].apply(score_to_category)
        
        # Add range
        quantiles = list(quantiles)
        df_tmp1 = df[df[f"{column}_cat"] == -1]
        df_tmp1[f"{column}_range"] = 'Data not available'
        df_tmp2 = df[df[f"{column}_cat"] != -1]
        df_tmp2[f"{column}_range"] = df_tmp2[f"{column}_cat"].apply(lambda x: str([quantiles[x],quantiles[x+1]]))
        
        df = pd.concat([df_tmp1, df_tmp2])
        
    return df

def compute_category_label(df, columns):
    df = df.copy()
    for column in columns:
        df.sort_values(column, inplace=True)
                                                          
        # Add categories
        df[f"{column}_cat"] = df[f"{column}"].apply(score_to_category)
        
        # Add range
        ranges = [0,1,2,3,4,5]
        df_tmp1 = df[df[f"{column}_cat"] == -1]
        df_tmp1[f"{column}_range"] = 'Data not available'
        df_tmp2 = df[df[f"{column}_cat"] != -1]
        df_tmp2[f"{column}_range"] = df_tmp2[f"{column}_cat"].apply(lambda x: str([ranges[x],ranges[x+1]]))
        
        df = pd.concat([df_tmp1, df_tmp2])
        
    return df

def compute_decile_category_label(df, columns):
    df = df.copy()
    for column in columns:
        # Add scores
        df[f"{column}_decile"] = df[column].apply(lambda x: int(np.round(x)))
                                                          
        # Add categories
        df[f"{column}_cat"] = df[f"{column}_decile"].apply(decile_to_category)
        
        # Add range
        ranges = [0,2,4,6,8,10]
        df_tmp1 = df[df[f"{column}_cat"] == -1]
        df_tmp1[f"{column}_range"] = 'Data not available'
        df_tmp2 = df[df[f"{column}_cat"] != -1]
        df_tmp2[f"{column}_range"] = df_tmp2[f"{column}_cat"].apply(lambda x: str([ranges[x],ranges[x+1]]))
        
        df = pd.concat([df_tmp1, df_tmp2])
        
    return df

def compute_percentile(df, columns):
    df = df.copy()
    q = [0,0.2,0.4,0.6,0.8,1]
    scores = [0,0.2,0.4,0.6,0.8,1]
    for column in columns:
        df.sort_values(column, inplace=True)
        
        s = df[column]
        f, quantiles =quantile_interp_function(s,q,scores)

        # Add percentile
        df[column.replace('EP',  'EPL')] = df[column].apply(f)                                                      
        
    return df

def set_lat_lon_attrs(ds):
    """ Set CF latitude and longitude attributes"""
    ds["lon"] = ds.lon.assign_attrs({
      'axis' : 'X',
       'long_name' : 'longitude',
        'standard_name' : 'longitude',
         'stored_direction' : 'increasing',
          'type' : 'double',
           'units' : 'degrees_east',
            'valid_max' : 360.0,
             'valid_min' : -180.0
             })
    ds["lat"] = ds.lat.assign_attrs({
      'axis' : 'Y',
       'long_name' : 'latitude',
        'standard_name' : 'latitude',
         'stored_direction' : 'increasing',
          'type' : 'double',
           'units' : 'degrees_north',
            'valid_max' : 90.0,
             'valid_min' : -90.0
             })
    return ds

def create_ds_mask(df, ds, name, lon_name='lon', lat_name='lat'):
    # Create index column
    if 'index' not in df:
        df = df.reset_index(drop=True).reset_index()
    
    # Get mean ds cell area (in degrees) 
    mean_y_size = np.diff(ds.lat.values).mean()
    #print(mean_y_size)
    mean_x_size = np.diff(ds.lat.values).mean()
    #print(mean_x_size)
    mean_area = mean_y_size * mean_x_size
    print(f"The mean ds cell area is {np.round(mean_area, 6)} deg.\n")
    
    # Clip gdf to bounding box of ds
    xmin = ds.lon.min().values.tolist()
    xmax = ds.lon.max().values.tolist()
    ymin = ds.lat.min().values.tolist()
    ymax = ds.lat.max().values.tolist()
    df = df.cx[xmin:xmax, ymin:ymax]
    
    
    # Add area of geoms to gdf
    df = df.assign(area = df.area)
    df = df.assign(area_is_gt_cell = df['area'] > mean_area)
    print(f"Clipped gdf to dataset bounds, giving {len(df['index'])} potential geometries, of which {df['area_is_gt_cell'].sum()} are large enough.\n")
    
    print("Geometries smaller than mean cell size:")
    print(df.loc[df['area_is_gt_cell'] == False, ['index']])
    print("\n")

    # Extract indexes and geoms that are large enough!
    id_ints = df.loc[df['area_is_gt_cell'] == True, 'index'].values
    geoms = df.loc[df['area_is_gt_cell'] == True, 'geometry'].values
    
    print(f'Number of indexes: {len(id_ints)}')
    print(f'Number of geoms: {len(geoms)}')

    # create mask object
    da_mask = regionmask.Regions(
      name = name,
      numbers = id_ints,
      outlines = geoms)\
      .mask(ds, lon_name=lon_name, lat_name=lat_name)\
      .rename(name)

    # get the ints actually written to mask
    id_ints_mask = da_mask.to_dataframe().dropna()[name].unique()
    id_ints_mask = np.sort(id_ints_mask).astype('int')
    
    print(f'Number of ints in mask: {len(id_ints_mask)}')
    
    # update da attributes
    da_mask.attrs['id_ints'] = id_ints_mask
    da_mask = set_lat_lon_attrs(da_mask)
    return da_mask, df[df['area_is_gt_cell'] == False]['index']

def rtree_intersect(gdf, geometries):
    number_fires = []
    mean_fire_size = []
    total_fire_size = []
    per_total_fire_size = []
    sindex = gdf.sindex
    # Iterate over the geometries
    for n, geometry in enumerate(tqdm(geometries.geometry)):
        # Find approximate matches with r-tree
        possible_matches_index = list(sindex.intersection(geometry.bounds))
        possible_matches = gdf.iloc[possible_matches_index]
        # Find precise matches with r-tree
        precise_matches = possible_matches[possible_matches.intersects(geometry)]
        precise_matches['geometry'] = precise_matches.intersection(geometry)
        
        if not precise_matches.empty:
            # Add area in ha
            precise_matches['area_ha'] = precise_matches['geometry'].to_crs({'init': 'epsg:32633'}).map(lambda p: p.area / 10**4)
            number_fires.append(len(precise_matches))
            mean_fire_size.append(precise_matches['area_ha'].mean())
            
            total_fire_size.append(cascaded_union(precise_matches['geometry'].to_crs({'init': 'epsg:32633'})).area / 10**4)
            per_total_fire_size.append(cascaded_union(precise_matches['geometry']).area/geometry.area*100)
        else:
            number_fires.append(0)
            mean_fire_size.append(0.)
            total_fire_size.append(0.)
            per_total_fire_size.append(0.)
    return number_fires, mean_fire_size, total_fire_size, per_total_fire_size